In [ ]:
from copy import deepcopy
import torch 
import torch.nn as nn
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re
import string
import nltk
from torch import optim
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
import plotly.figure_factory as ff
from textblob import TextBlob
import numpy as np
from tqdm import tqdm
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from scipy.sparse import load_npz
from torch.utils.data import Dataset, DataLoader
from TextDataset import TextDataset
from DualArchitecture import DualTextCNN
import gensim.downloader as api

def get_sentence_embedding(sentence):
    words = sentence.split()
    word_embeddings = [embedding_model[word] for word in words if word in embedding_model]
    if len(word_embeddings) == 0:
        return np.zeros(embedding_model.vector_size)
    return np.mean(word_embeddings, axis=0)


BEST_MODEL_SAVE_PATH = "C:/Users/Korhan/Desktop/workspace/vsCodeWorkspace/Python_Workspace/mental_health_sentiment_analysis/latentG_loss_best_dualtextcnnModel.pt"
torch.cuda.empty_cache()
torch.manual_seed(42)
torch.cuda.manual_seed(42)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

texts = []

text1 = "first posted back years pandemic later years old tbh still struggling lot lash anymore self harm really abuse alcohol drugs various things feel something like used emotional wreck back emotionless one sometimes overwhelming always constant feelings loneliness never left feel nowadays even though used fucked shit still felt something whether pain sadness whole range emotions feel empty decided change life take path inevitably leading suicide sure think id today insecurities alone one cares whether family friends grown massively point sorta accepted going post wrote back said afraid eventually pushing everyone away afraid gunna leave people know see better longer guy would get black drunk end hurting ones around tbh sorted issues better hiding even better want die trust know never want go back want feel like thing idea back start clear issues could deal better eg abuse alcohol every opportunity different issues painful make sad know head real honestly even know anymore think pretty good picking things coupled fucked thoughts head create scenarios illusions gets harder harder tell real ultimately guess much pain hurt automatically accepted alone one truly cares end suffering sucks lot sometimes wonder life better back fucked unstable care loves whatever plan living much longer actually successfully killed none would matter suicide option like want feel better feel loved see light end tunnel empty lonely darkness lost soul"
# Personality disorder

text2 = "trouble sleeping, confused mind, restless heart. all out of tune" 
# Anxiety

text3 = "thought wanting exist much denial spoken others desire stop existing know also make joke occasion yet vent like annoying people around pushing away want hear problem want happy strong pillar always listens need way around spoilered nsfw one triggering two like le people see profile vented one aspect thought shared space channel triggering topic think want new outlet like relying reddit anything putting sensitive information good chance deleting post later feel like might triggering keep thinking sensory deprivation void dark grey stimulus without need sustenance start fade brain activity would shut sort isolation keep wanting offending sound smell need eat bodily function pain going without pain brain suffering ill effect current lifestyle interesting artistic vision see much share possibly triggering artistic vision interesting viewed lens say le concerned making sense anymore used put much effort syntax came across sick trying hard people good rambling writing style using lately tired putting effort say thing ashamed weird people good alright suffering bad issue go away sometimes hard time keeping appearance ashamed much neurotic shared okay okay accepting going allow feel hate emotion like usual okay"
# Stress

text4 = "oh no im restless the time will come i walk with jesus. then my heart is calm every time i sing a song i dont know tomorrow in this section i always want to cry because i sing it but i dont do it sorry god i worry too much ️"
# Stress

text5 = "life is too hard to bare i cant take it anymore."
# Suicidal

texts.append(text1)
texts.append(text2)
texts.append(text3)
texts.append(text4)
texts.append(text5)


id2label = {
    0: "Anxiety", 
    1: "Depression", 
    2: "Stress", 
    3: "Suicidal",
    4: "Normal",
    5: "Bipolar",
    6: "Personality disorder",
}
label2id = {v: k for k, v in id2label.items()} 


embedding_model = api.load("word2vec-google-news-300")
embeddings = np.array([get_sentence_embedding(sentence) for sentence in texts])
print(embeddings.shape)

(5, 300)


In [2]:
X_tensor = torch.tensor(embeddings).float().unsqueeze(1).to("cuda")

In [3]:
model = DualTextCNN(input_dim=300, num_classes=7, latent_dim=32).to(DEVICE)
model.load_state_dict(torch.load(BEST_MODEL_SAVE_PATH, map_location=torch.device(DEVICE)))

<All keys matched successfully>

In [4]:
model.eval()
with torch.no_grad():
    y_pred, _, _ = model(X_tensor)
    predicted_ids = torch.argmax(y_pred, dim=1)
    predicted_labels = [id2label[id.item()] for id in predicted_ids]
    print("Predicted ids : ", predicted_ids.cpu().numpy())
    print("Predicted class labels: ", predicted_labels)

Predicted ids :  [6 0 2 2 3]
Predicted class labels:  ['Personality disorder', 'Anxiety', 'Stress', 'Stress', 'Suicidal']


In [ ]:
# "Personality Disorder",
# "Anxiety",
# "Stress", or "Depression"
# "Stress",
# "Suicidal",